# Visualizing Spatial Information - California Housing

This demo shows a simple workflow when working with geospatial data:

 * Obtaining a dataset which includes geospatial references.
 * Obtaining a desired geometries (boundaries etc.)
 * Visualisation 
 
In this example we will make a simple **proportional symbols map** using the `California Housing` dataset in `sklearn` package.

In [1]:
import pandas as pd
from lets_plot import *

LetsPlot.setup_html()

import lets_plot
lets_plot.__version__

'4.5.1'

## The Housing Data

In [2]:
from sklearn.datasets import fetch_california_housing

california_housing_bunch = fetch_california_housing()
data = pd.DataFrame(california_housing_bunch.data, columns=california_housing_bunch.feature_names)

# Add $-value field to the dataframe.
# dataset.target: numpy array of shape (20640,)
# Each value corresponds to the average house value in units of 100,000.
data['Value($)'] = california_housing_bunch.target * 100000
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Value($)
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,452600.0
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,358500.0
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,352100.0
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,341300.0
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,342200.0


In [3]:
# Draw a random sample from the data set.
data = data.sample(n=1000)

## California Polygon (GeoDataframe)

We will use Lets-Plot geocoding API to obtain California boundaries.

In [4]:
# CA = gpd.read_file("./ca-state-boundary/CA_State_TIGER2016.shp")

from lets_plot.geo_data import *

CA = geocode_states('CA').scope('US').inc_res(2).get_boundaries()
CA.head()

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


,state,found name,geometry
0,CA,California,"MULTIPOLYGON (((-124.32694 40.6162, -124.31188..."


## None-interactive map

Use regular `ggplot2` geometries to create none-interactive proportional symbol map.

In [5]:
# The plot base 
p = ggplot()\
  + theme(axis='blank', panel_grid='blank')\
  + scale_fill_brewer(palette="PiYG", direction=-1)\
  + scale_size(range=[5, 15], guide='none')\
    

# The points layer
points = geom_point(aes(x='Longitude',
                        y='Latitude',
                        size='Value($)',
                        fill='HouseAge'), 
                    data=data,
                    shape = 24, color = "white",
                    alpha=0.8)

# The map
p + geom_polygon(data=CA, fill='#F8F4F0', color='#B71234')\
  + points\
  + ggsize(650, 600)

## Interactive map

To make map interactive just add an interactive basemap layer with the help of the `geom_livemap()` function.

### Configuring map tiles (optional)

By default *Lets-PLot* offers high quality vector map tiles but also can fetch raster tiles from a 3d-party Z-X-Y [tile servers](https://wiki.openstreetmap.org/wiki/Tile_servers).

For the sake of the demo lets use *CARTO Antique* tiles by [CARTO](https://carto.com/attribution/) as our basemap.

In [6]:
from lets_plot import tilesets

In [7]:
# LetsPlot.set(
#     maptiles_zxy(
#         url='https://cartocdn_c.global.ssl.fastly.net/base-antique/{z}/{x}/{y}@2x.png',
#         attribution='<a href="https://www.openstreetmap.org/copyright">© OpenStreetMap contributors</a> <a href="https://carto.com/attributions#basemaps">© CARTO</a>, <a href="https://carto.com/attributions">© CARTO</a>'
#     )
# )
LetsPlot.set(tilesets.CARTO_ANTIQUE_HIRES)

### Create interactive proportional symbol map

In [8]:
(p + geom_livemap()
   + geom_polygon(data=CA, fill='white', color='#B71234', alpha=0.5, size=0.3)
   + points
   + ggsize(750, 600))

WARN: The tileset is no longer available and the corresponding constant will be removed in future releases.


### Adjust the initial viewport

Use parameters `location` and `zoom` to define the initial viewport.

In [9]:
# Use `location` and `zoom` parameters to focus on Los Angeles area.
(p + geom_livemap(location=[-118.15, 33.96], zoom=7)
   + geom_polygon(data=CA, fill='white', color='#B71234', alpha=0.5, size=.7)
   + points)

WARN: The tileset is no longer available and the corresponding constant will be removed in future releases.
